# Tree search

## Code

In [89]:
# -*- coding: utf-8 -*-
"""
Editor de Spyder

Este es un archivo temporal.
"""

import typing
import math
import collections
import dataclasses
import abc
import itertools

T = typing.TypeVar("T", int, float)
Vector = typing.Tuple[T, T]
VectorComponent = typing.List[T]
Vertex = typing.List[typing.Tuple[T, T]]

def check_axis_intersect(that: Vector, z: T):
    x, y = that
    return y >= z >= x


def is_collision(that: Vertex, other: VectorComponent) -> bool:
    if len(that) != len(other):
        raise ValueError("Invalid arguments, different length of values")

    return all([check_axis_intersect(t, z) for t, z in zip(that, other)])


@dataclasses.dataclass
class VectorNode:
    data: VectorComponent
    value: T

    def __gt__(self, other):
        return self.value > other.value

    def __str__(self):
        return f"{self.data}, {self.value}"

    def __iter__(self):
        return iter([*self.data, self.value])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        return self.vertex[item]

    def __setitem__(self, key, value):
        self.vertex[key] = value


def calc_distance(x: T, y: T):
    return math.fabs(y - x) / 2


def create_axis(_array: Vertex):
    all_data = []

    for arr in _array:
        d = calc_distance(*arr)

        all_data.append(((x, x + d), (x + d, y)))

    return all_data


def create_axis_node(arrays):
    return itertools.product(arrays)


@dataclasses.dataclass
class Node:
    vertex: Vertex
    nodes: list

    def append(self, node: VectorComponent):
        self.nodes.append(node)

    def is_collide(self, node: VectorComponent):
        return is_collision(self.vertex, node)

    def __len__(self):
        return len(self.nodes)

    def __iter__(self):
        return iter(self.nodes)

    def __hash__(self):
        return hash(
            tuple(self.vertex)
        )


@dataclasses.dataclass
class NodeValue:
    vertex: Vertex
    node: VectorNode

    def __gt__(self, other: VectorNode):
        return self.value > other.value

    @property
    def value(self):
        return self.node.value


class NodeContainerInterface(abc.ABC):
    @property
    @abc.abstractmethod
    def axis(self) -> Vertex: ...

    @property
    @abc.abstractmethod
    def children(self): ...

    @property
    @abc.abstractmethod
    def node(self) -> Node: ...

    @property
    @abc.abstractmethod
    def is_parent(self) -> bool: ...

    @abc.abstractmethod
    def insert(self, verx: VectorComponent): ...

    @abc.abstractmethod
    def sort(self): ...

    @abc.abstractmethod
    def __iter__(self): ...


class TreeNode:
    def __init__(self, axis: Vertex, limit_divisions=1):
        self.__children = set()
        self.__node = Node(vertex=axis, nodes=[])
        self.__axis = list(axis)
        self.__limit_divisions = limit_divisions
        
        """
        if limit_divisions > 0:
            self.__children = [
                TreeNode(vertices, limit_divisions - 1) for vertices in create_axis(data)
            ]
        """

    def __hash__(self):
        return hash(
            tuple(self.__axis)
        )
    
    @property
    def axis(self):
        return [*self.__axis]

    @property
    def children(self):
        return self.__children

    @property
    def node(self) -> Node:
        return self.__node

    @property
    def is_parent(self):
        return len(self.children) != 0

    def insert(self, verx: VectorComponent):
        return self._insert_recursive(verx)

    def sort(self):
        return self._get_iter_child_recursive()

    def __iter__(self):
        return iter(self._get_iter_child_recursive())
        
    def _insert_recursive(self, verx: VectorComponent):
        def create_vertex(verx):
            root_axis = collections.deque()
            
            for axis, c in zip(self.axis, verx):
                x, y = axis
                d = calc_distance(x, y)
                
                print(d)
                
                if c > x + d:
                    root_axis.append((x, x + d))
                else:
                    root_axis.append((x + d, y))

            return root_axis

        tree = TreeNode(create_vertex(verx), self.__limit_divisions - 1)

        if tree in self.children:
            tree = list(self.children).index(tree)
        else:
            self.__children.add(tree)

        if self.__limit_divisions > 0:
            return tree.insert(verx)
        else:
            tree.node.append(verx)
        
        print(tree.node)
        
        return tree

    def _get_iter_child_recursive(self):
        def get_iter_child(root, nodes=None):
            if nodes is None:
                nodes = []

            # print(root.children)
            
            for child in list(root.children):
                node = child.node
                
                if child.is_parent:
                    get_iter_child(child, nodes)
                else:
                    if len(child.node) > 0:
                        nodes.append(child.node)

            return nodes

        return get_iter_child(self, [])


In [90]:
axis = (0, 1), (0, 1)
tree = TreeNode(axis)
tree.insert((.1, .1)).node
tree.insert((.1, .2)).node

tree.sort()

0.5
0.5
0.25
0.25
Node(vertex=deque([(0.75, 1), (0.75, 1)]), nodes=[(0.1, 0.1)])
0.5
0.5
0.25
0.25
Node(vertex=deque([(0.75, 1), (0.75, 1)]), nodes=[(0.1, 0.2)])


[Node(vertex=deque([(0.75, 1), (0.75, 1)]), nodes=[(0.1, 0.2)]),
 Node(vertex=deque([(0.75, 1), (0.75, 1)]), nodes=[(0.1, 0.1)])]

## Initialize and config data

In [2]:
import sympy as sp
import numpy as np

N_COMPONENTS = 6

syms = sp.symbols(f'x1:{N_COMPONENTS + 1}')

ALL_SYMS = syms
LIMITS_RANGE = -3, 3

def get_n_syms(_range=(-1, 1)):
    return [_range for _ in range(len(syms))]

In [3]:
import operator
import functools

x1 = syms[0]
x2 = syms[1]
x3 = syms[2]

eq = sum(syms)
# eq = eq.subs(x1, 1/x1**2).subs(x2, x2**2).subs(x3, x3**3)
# eq = functools.reduce(operator.mul, syms, 1)**2 / 100

OBJ_FUNC = sp.lambdify(syms, eq, modules='numpy')

eq

x1 + x2 + x3 + x4 + x5 + x6

## Execution

In [4]:
import collections
import functools
import itertools

import pandas as pd

def tree_search_algorithm(f, axis, n_it=10, n_p=5, n_limit=0):
    tree = NodeContainer(axis, 2)

    all_it = collections.deque()
    all_it_route = collections.deque()

    for it in range(n_it):
        local_min_point = collections.deque()

        for _ in range(n_p):
            data = [np.random.uniform(*_) for _ in tree.axis]
            value = f(*data)

            tree.insert(VectorNode(data, value))

        for container in tree.sort():
            min_node = min(container)

            local_min_point.append(
                NodeValue(container.vertex, min_node)
            )

        if len(local_min_point) <= 0:
            continue

        local_min_node = min(local_min_point)

        all_it.append(list(min(local_min_point).node))
        all_it_route.append(local_min_node.vertex)

        tree = NodeContainer(local_min_node.vertex, 2)

    return all_it, all_it_route


all_it, all_it_route = tree_search_algorithm(
    OBJ_FUNC, [(-3, 3) for _ in range(len(syms))], 5, 5
)


In [5]:
pd.DataFrame(all_it_route)

,0,1,2,3,4,5
0,"(1.5, 3)","(-3, -1.5)","(-3, -1.5)","(0.0, 1.5)","(-3, -1.5)","(-1.5, 0.0)"
1,"(1.5, 1.875)","(-1.875, -1.5)","(-3, -2.625)","(0.375, 0.75)","(-2.25, -1.875)","(-0.375, 0.0)"
2,"(1.78125, 1.875)","(-1.875, -1.78125)","(-2.90625, -2.8125)","(0.375, 0.46875)","(-2.0625, -1.96875)","(-0.28125, -0.1875)"
3,"(1.78125, 1.8046875)","(-1.8515625, -1.828125)","(-2.8359375, -2.8125)","(0.421875, 0.4453125)","(-2.0625, -2.0390625)","(-0.234375, -0.2109375)"
4,"(1.78125, 1.787109375)","(-1.8515625, -1.845703125)","(-2.830078125, -2.82421875)","(0.427734375, 0.43359375)","(-2.0625, -2.056640625)","(-0.216796875, -0.2109375)"


In [6]:
pd.DataFrame(all_it)

,0,1,2,3,4,5,6
0,2.341962,-2.585922,-2.994928,0.325170,-1.751539,-1.155528,-5.820786
1,1.657842,-1.698363,-2.844942,0.496560,-1.923584,-0.342788,-4.655275
2,1.851800,-1.850503,-2.818561,0.431978,-2.059606,-0.241305,-4.686197
3,1.794385,-1.846314,-2.821549,0.439056,-2.044100,-0.229292,-4.707814
4,1.785113,-1.851320,-2.824652,0.429681,-2.060933,-0.213136,-4.735247
